In [1]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.tuner import IntegerParameter, ContinuousParameter, CategoricalParameter, HyperparameterTuner
from sagemaker import get_execution_role
import os

print("--- ⚙️ Step 1: Initializing SageMaker Environment ---")

# 1. 获取执行角色 (IAM Role)
# 如果在本地 VS Code 运行，可能需要手动指定 Role ARN
try:
    role = get_execution_role()
except ValueError:
    print("⚠️ 警告: 无法自动获取角色，请手动填入 ARN。")
    # 请去 AWS Console -> IAM -> Roles 找一个类似 AmazonSageMaker-ExecutionRole 的角色
    role = "arn:aws:iam::137568342316:role/SageMakerExecutionRole" 

# 2. 基础配置
sagemaker_session = sagemaker.Session()
bucket_name = 'sleep-disorder-mlops-bucket' # 您的 S3 桶名

# 3. 指定代码源 (Source of Truth for Code)
# SageMaker 会自动 `git clone` 这个仓库到训练实例中
git_repo = 'https://github.com/vinnie071015/sleeping-disorder-mlops.git' # 替换为您的仓库地址
git_config = {
    'repo': git_repo, 
    'branch': 'main'
}

print(f"✅ Role: {role}")
print(f"✅ Bucket: {bucket_name}")
print(f"✅ Git Repo: {git_repo}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/zhengchengsheng/Library/Application Support/sagemaker/config.yaml
--- ⚙️ Step 1: Initializing SageMaker Environment ---


Couldn't call 'get_role' to get Role ARN from role name ML_Project to get Role path.


⚠️ 警告: 无法自动获取角色，请手动填入 ARN。
✅ Role: arn:aws:iam::137568342316:role/SageMakerExecutionRole
✅ Bucket: sleep-disorder-mlops-bucket
✅ Git Repo: https://github.com/vinnie071015/sleeping-disorder-mlops.git


In [2]:
print("\n--- 💾 Step 2: Defining S3 Data Input ---")

# 定义 S3 数据输入
# SageMaker 会自动把这个 S3 路径下的所有文件下载到容器内的 /opt/ml/input/data/train/
# 这里的 s3_data 必须指向包含 sleep_data.csv 的文件夹路径 (以 / 结尾)
s3_input_path = f's3://{bucket_name}/raw_data/'

train_input = sagemaker.inputs.TrainingInput(
    s3_data=s3_input_path, 
    content_type='text/csv'
)

print(f"✅ Training Data Source: {s3_input_path}")


--- 💾 Step 2: Defining S3 Data Input ---
✅ Training Data Source: s3://sleep-disorder-mlops-bucket/raw_data/


In [3]:
print("\n--- 🏗️ Step 3: Defining Common Estimator Configuration (Git Mode) ---")

# 使用 ml.m5.large (通用型)
instance_type = 'ml.m5.large'

common_estimator_args = {
    # 修正 1: 入口文件路径要包含 src (因为 source_dir 变成了根目录)
    'entry_point': 'src/train.py',
    
    # 修正 2: source_dir 设为 '.' (代表 Git 仓库的根目录)
    # 这样 SageMaker 能同时看到 src/ 文件夹和 requirements.txt
    'source_dir': '.',
    
    'role': role,
    'instance_count': 1,
    'instance_type': instance_type,
    'framework_version': '1.2-1',
    'py_version': 'py3',
    'git_config': git_config,
    'sagemaker_session': sagemaker_session,
    
    # 修正 3: 不需要手动指定 dependencies 了
    # 因为 requirements.txt 就在 source_dir (根目录) 下，SageMaker 会自动识别并安装
    # 'dependencies': [...] <--- 删除这行
}

print(f"✅ Instance Type: {instance_type}")
print(f"✅ Path Correction: source_dir='.', entry_point='src/train.py'")


--- 🏗️ Step 3: Defining Common Estimator Configuration (Git Mode) ---
✅ Instance Type: ml.m5.large
✅ Path Correction: source_dir='.', entry_point='src/train.py'


In [4]:
print("\n--- 🎛️ Step 4: Defining Tuners for 3 Models ---")

# 定义指标抓取规则 (对应 src/train.py 中的 print 语句)
metric_definitions = [
    {'Name': 'accuracy', 'Regex': '✅ Accuracy: ([0-9\\.]+)'},
    {'Name': 'f1', 'Regex': '✅ F1 Score: ([0-9\\.]+)'}
]

# ==========================================
# A. Random Forest Tuner
# ==========================================
rf_estimator = SKLearn(**common_estimator_args)
# 固定模型类型为 RF
rf_estimator.set_hyperparameters(model_type='random_forest')

rf_tuner = HyperparameterTuner(
    estimator=rf_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'n_estimators': IntegerParameter(50, 150),
        'max_depth': IntegerParameter(5, 15)
    },
    max_jobs=2,          # 总共跑 2 次 (省钱)
    max_parallel_jobs=1, # 串行跑 (安全)
    base_tuning_job_name='rf-tuning'
)

# ==========================================
# B. SVM Tuner
# ==========================================
svm_estimator = SKLearn(**common_estimator_args)
svm_estimator.set_hyperparameters(model_type='svm')

svm_tuner = HyperparameterTuner(
    estimator=svm_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'C': ContinuousParameter(0.1, 5.0),
        'kernel': CategoricalParameter(['rbf', 'linear'])
    },
    max_jobs=2,
    max_parallel_jobs=1,
    base_tuning_job_name='svm-tuning'
)

# ==========================================
# C. Logistic Regression Tuner
# ==========================================
lr_estimator = SKLearn(**common_estimator_args)
lr_estimator.set_hyperparameters(model_type='logistic_regression')

lr_tuner = HyperparameterTuner(
    estimator=lr_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'C': ContinuousParameter(0.1, 5.0)
    },
    max_jobs=2,
    max_parallel_jobs=1,
    base_tuning_job_name='lr-tuning'
)

print("✅ Tuners for RF, SVM, and LR are ready.")


--- 🎛️ Step 4: Defining Tuners for 3 Models ---


/Users/zhengchengsheng/opt/anaconda3/envs/mlops-env/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


✅ Tuners for RF, SVM, and LR are ready.


In [14]:
import time

print("\n--- 🚀 Step 5: Launching Jobs Sequentially ---")

# 1. 运行 Random Forest
print(f"\n🌲 [1/3] Launching Random Forest Tuning... (Timestamp: {time.strftime('%H:%M:%S')})")
# SageMaker 会自动生成唯一的 Job Name，这里 fit 会直接打印出来
rf_tuner.fit({'train': train_input}, wait=True) 
print("✅ Random Forest Tuning Completed!")

# 2. 运行 SVM
print(f"\n🛡️ [2/3] Launching SVM Tuning... (Timestamp: {time.strftime('%H:%M:%S')})")
svm_tuner.fit({'train': train_input}, wait=True)
print("✅ SVM Tuning Completed!")

# 3. 运行 Logistic Regression
print(f"\n📈 [3/3] Launching Logistic Regression Tuning... (Timestamp: {time.strftime('%H:%M:%S')})")
lr_tuner.fit({'train': train_input}, wait=True)
print("✅ Logistic Regression Tuning Completed!")

print("\n🎉 All experiments finished!")


--- 🚀 Step 5: Launching Jobs Sequentially ---

🌲 [1/3] Launching Random Forest Tuning... (Timestamp: 10:38:31)


Cloning into '/var/folders/py/wsw9dv2n5g70rl8sm3pxxjg40000gn/T/tmphzjps5_c'...
Already on 'main'


Your branch is up to date with 'origin/main'.


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


.....................................................*


UnexpectedStatusException: Error for HyperParameterTuning job rf-tuning-251203-1038: Failed. Reason: All training jobs failed. Please take a look at the training jobs failures to get more details.. Check troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html

In [8]:
import time
import sys
import boto3

# ------------------------------------------------------------------
# ⚠️ 确保在运行这段代码之前，前面的配置代码块已经成功执行。
# ------------------------------------------------------------------

# --- 运行 Random Forest Estimator (单次 Fit) ---

# 重新生成一个唯一的 Job Name，确保每次都是新任务
TRAINING_JOB_NAME = f"log-test-full-deps-{time.strftime('%Y%m%d-%H%M%S')}"

print("\n--- 🚀 Step 5: Launching Full Dependencies Test Job ---")
print(f"🌲 [1/1] Launching Test Training Job... (Timestamp: {time.strftime('%H:%M:%S')})")
print(f"   -> 任务名: {TRAINING_JOB_NAME}")
print("   -> 模式: 完整 requirements.txt + 极简 train.py")


try:
    # 使用 rf_estimator 对象，但直接调用 fit，绕过 HyperparameterTuner
    rf_estimator.fit(
        inputs={'train': train_input}, 
        job_name=TRAINING_JOB_NAME, 
        wait=False # 禁用阻塞，我们使用自定义循环
    )
    
except Exception as e:
    print(f"\n❌ 任务启动失败: {e}")
    sys.exit(1)


# --- 状态监控循环 ---
print(f"\n✅ 任务已提交。开始实时监控状态...")
sm_client = rf_estimator.sagemaker_session.sagemaker_client

while True:
    time.sleep(15) # 每 15 秒查询一次
    try:
        # 获取最新的训练任务状态
        desc = sm_client.describe_training_job(TrainingJobName=TRAINING_JOB_NAME)
        status = desc['TrainingJobStatus']
        
        # 打印当前状态
        secondary_status = desc.get('SecondaryStatus', 'N/A')
        print(f"\r[状态更新] 任务状态: {status:<10} | 容器状态: {secondary_status:<15}", end="", flush=True)
        
        if status in ['Completed', 'Failed', 'Stopped']:
            print(f"\n--- 任务结束，最终状态: {status} ---")
            
            if status == 'Failed':
                failure_reason = desc.get('FailureReason', '无顶层原因')
                print(f"💥 失败原因 (API Message): {failure_reason}")
                
                # 提示用户手动检查 CloudWatch 日志
                print("\n⚠️ 注意: 任务失败，请手动在 CloudWatch 控制台搜索该任务名称，查看 pip 安装的错误。")

            break
            
    except Exception as e:
        print(f"\n❌ 状态查询失败: {e}")
        break

print("\n🎉 诊断任务执行完毕！")


--- 🚀 Step 5: Launching Full Dependencies Test Job ---
🌲 [1/1] Launching Test Training Job... (Timestamp: 14:35:43)
   -> 任务名: log-test-full-deps-20251203-143543
   -> 模式: 完整 requirements.txt + 极简 train.py


Cloning into '/var/folders/py/wsw9dv2n5g70rl8sm3pxxjg40000gn/T/tmpm5nmei56'...
Already on 'main'


Your branch is up to date with 'origin/main'.


INFO:sagemaker:Creating training-job with name: log-test-full-deps-20251203-143543



✅ 任务已提交。开始实时监控状态...
[状态更新] 任务状态: Completed  | 容器状态: Completed      
--- 任务结束，最终状态: Completed ---

🎉 诊断任务执行完毕！
